In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
root = "https://genshin-impact.fandom.com/wiki/Wish/History"
rC = requests.get(root)

soup = BeautifulSoup(rC.content, "html.parser")

charDict = {}

banner_table = soup.select(".article-table")

char_5star = []
char_date = []
standard_5star = []
version = []
version_date = []

# char_4star = []

for ver_ind, link in enumerate(soup.select('a[title*="Version"]')):
    version.append(link.get_text().strip())
    v_date = set()
    for table_ind, table_row in enumerate(
        banner_table[ver_ind].select("tbody")[0].select("tr")
    ):
        if table_ind > 0:
            target_chars = table_row.select("td")[1].select(".card-quality-5")
            card_class = target_chars[0].get("class")
            banner_title = table_row.select("td")[0].get_text()
            banner_dates = table_row.select("td")[2].get_text().strip().split(" – ")

            # banner_start_date = banner_dates[0]
            # banner_end_date = banner_dates[1]

            if len(card_class) == 4:
                for bd in banner_dates:
                    v_date.add(bd)
                for chars in target_chars:
                    if len(chars.get("class")) == 4:
                        c5s = chars.select("a")[0].get("title").strip()

                        if banner_title.strip() != "Wanderlust Invocation":
                            if c5s not in char_5star:
                                char_5star.append(c5s)
                                char_date.append([banner_dates])
                            else:
                                char_ind = char_5star.index(c5s)
                                char_date[char_ind].append(banner_dates)
                        else:
                            standard_5star.append(c5s)
    version_date.append(list(v_date))

for ci, c in enumerate(char_5star):
    if c in standard_5star:
        char_5star[ci] = c + "*"

In [3]:
def current_banner(date1, date2):
    if (
        datetime.strptime(date1, "%B %d, %Y").date() < datetime.today().date()
        and datetime.strptime(date2, "%B %d, %Y").date() > datetime.today().date()
    ):
        return True


def last_banner(last_date):
    target_date = datetime.strptime(last_date, "%B %d, %Y").date()
    difference = datetime.today().date() - target_date

    return difference.days

In [4]:
last_ver_list = []
last_run_list = []

for c in range(len(char_5star)):
    if char_date[c][-1][0] == "TBA" or char_date[c][-1][-1] == "TBA":
        last_run = (
            int(last_banner(char_date[c][-1][0]))
            if char_date[c][-1][0] != "TBA"
            else -999
        )
        last_run_list.append(last_run)
    elif char_date[c][-1][-1] == "Indefinite":
        last_run = char_date[c][-1][0]
        last_run_list.append(int(last_banner(last_run)))
    else:
        last_run = char_date[c][-1][-1]
        last_run_list.append(int(last_banner(last_run)))
        if current_banner(char_date[c][-1][0], char_date[c][-1][-1]) == True:
            this_banner = version[banner_index]

    banner_index = next(
        (i for i, sublist in enumerate(version_date) if last_run in sublist), -1
    )
    last_ver_list.append(version[banner_index])


zipped_lists = list(
    zip(
        char_5star,
        char_date,
        last_ver_list,
        last_run_list,
    )
)

In [ ]:
sorted_lists = sorted(zipped_lists, key=lambda x: x[-1], reverse=True)
for ind, val in enumerate(sorted_lists):
    if val[2] == this_banner:
        print("{:20s} {:15s} {:>8}".format(val[0], val[2], "Ongoing"))
    elif val[3] > 0:
        print("{:20s} {:15s} {:>8}".format(val[0], val[2], val[3]))
    else:
        print("{:20s} {:15s} {:>8}".format(val[0], val[2], "OTW"))

In [ ]:
import json

title = ["Name", "Banner Dates", "Last Version", "Last Rerun"]
k = [
    dict(zip(title, row))
    for row in list(
        zip(
            char_5star,
            char_date,
            last_ver_list,
        )
    )
]

with open("new_charlist.json", "w") as fp:
    json.dump(k, fp)